# Building a new sensor
1. Define a specific sequence
2. Divide the sequence in windows of the same length of the trigger (36nt)
3. For each possible trigger, define a toehold sensor sequence
4. Filter sequences that contain stop codons
5. Evaluate sequence properties using NUPACK and rank them

### Import dependencies

In [1]:
import os
import os.path
from subprocess import Popen, PIPE
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import statsmodels.api as sm
from skbio import Sequence

/home/anibal/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Define library functions

In [2]:
def reversed_complement(sequence):
    mapping = {'A': 'U', 'G': 'C', 'U': 'A', 'C': 'G'}
    sequence_upper = sequence.upper()

    complement = ''
    for c in sequence_upper:
        complement += mapping[c]

    # reverse the sequence
    return complement[::-1]

def split_sequence(sequence, window):
    sequences = []
    limit = len(sequence) - window + 1

    for i in range(0, limit):
        sequences.append(sequence[i:window + i])

    return sequences

def no_stop(sequence):
    stop = ['UAA', 'UAG', 'UGA']

    for i in range(0, len(sequence), 3):
        if sequence[i:i + 3] in stop:
            return False

    return True

def possible_toehold_B(reg, rev):
    
    #to avoid any posterior conflicts please write loops and linkers in RNA form!
    
    loop = 'GGACUUUAGAACAGAGGAGAUAAAGAUG' # The last three nuc is the last three nucleotides are AUG, the start codon.
    
    linker = 'ACCUGGCGGCAGCGCAAGAAG'#( from Green paper 2019 se, NOTE THAT DOES NOT HAVE A MOCLO SITE C,  AATG)
    #linker = "AACCUGGCGGCAGCGCAAGAAGAUGCGUAAA" 
    toeholds = []

    for n in ['A', 'G', 'U', 'C']: 
        if no_stop(reg[0:11] + linker):
            toeholds.append(rev + loop + reg[0:11] + linker)
            return toeholds
      
    
    toeholds.append("STOP")
    return toeholds


### Step 1: Define a target sequence
In this case, we use the  5' of the glycoprotein 1 from PVY:

In [3]:
#seq = "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATG"
#seq= "CTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTA"
seq= "GGAGTTTGGGTTATGATGGATGGAGATGAACAAGTCGAATACCCACTGAAACCAATCGTTGAGAATGCAAAACCAACACTTAGGCAAATCATGGCACATTTCTCAGATGTTGCAGAAGCGTATATAGAAATGCGCAACAAAAAGGAACCATATATGCCACGATATGGTTTAGTTCGTAATCTGCGCGATGGAAGTTTGGCTCGCTATGCTTTTGACTTTTATGAAGTTAC"

Convert to RNA and determine the reverse complement

In [4]:
processed_sequence = seq.upper().replace('T', 'U') #/.replace(' ', '')
rc = reversed_complement(processed_sequence)
len(split_sequence(rc,36))

195

### Step 2: Determine 36-nucleotide sub-sequences
To do this, we make all possible triggers for the direct and reversed complementary sequence

In [5]:
d_1 = {'Triggers': split_sequence(processed_sequence,36)}
df_1 = pd.DataFrame(data=d_1)
df_1["Sense"]="Direct"

d_2 = {'Triggers': split_sequence(rc,36)}
df_2 = pd.DataFrame(data=d_2)
df_2["Sense"]="Reversed Complement"
frames = [df_1, df_2]
result = pd.concat(frames)
result

,Triggers,Sense
0,GGAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUC,Direct
1,GAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCG,Direct
2,AGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGA,Direct
3,GUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAA,Direct
4,UUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAU,Direct
5,UUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUA,Direct
6,UGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUAC,Direct
7,GGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACC,Direct
8,GGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCC,Direct
9,GUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCCA,Direct


### Step 3: For each trigger, design a toehold sensor

In [6]:
toeholds=[]
for i in range(len(result.iloc[:,0])):
    toeholds.append((possible_toehold_B(result.iloc[i,0],reversed_complement(result.iloc[i,0])))[0])
a=pd.Series(toeholds)
result["Toehold Switch"]=a.values
result

,Triggers,Sense,Toehold Switch
0,GGAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUC,Direct,GACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCCGGACUUUAGA...
1,GAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCG,Direct,CGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCGGACUUUAGA...
2,AGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGA,Direct,STOP
3,GUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAA,Direct,UUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACGGACUUUAGA...
4,UUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAU,Direct,STOP
5,UUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUA,Direct,UAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAGGACUUUAGA...
6,UGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUAC,Direct,GUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAGGACUUUAGA...
7,GGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACC,Direct,STOP
8,GGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCC,Direct,GGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCGGACUUUAGA...
9,GUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCCA,Direct,UGGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACGGACUUUAGA...


### Step 4: Remove sensors with STOP codons

In [7]:
df = result[result.iloc[:,2] != "STOP"]
df

,Triggers,Sense,Toehold Switch
0,GGAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUC,Direct,GACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCCGGACUUUAGA...
1,GAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCG,Direct,CGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCGGACUUUAGA...
3,GUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAA,Direct,UUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACGGACUUUAGA...
5,UUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUA,Direct,UAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAGGACUUUAGA...
6,UGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUAC,Direct,GUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAGGACUUUAGA...
8,GGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCC,Direct,GGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCGGACUUUAGA...
9,GUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCCA,Direct,UGGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACGGACUUUAGA...
12,AUGAUGGAUGGAGAUGAACAAGUCGAAUACCCACUG,Direct,CAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUGGACUUUAGA...
14,GAUGGAUGGAGAUGAACAAGUCGAAUACCCACUGAA,Direct,UUCAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUCGGACUUUAGA...
15,AUGGAUGGAGAUGAACAAGUCGAAUACCCACUGAAA,Direct,UUUCAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUGGACUUUAGA...


### Step 5: Evaluate sequence properties using NUPACK 

Scoring functions from Green (2019):
- Three-parameter fit (R2 = 0.57):
- Fold change = –71.7 dfull_sensor  – 49.1 dactive_sensor – 22.6 dbinding_site + 54.3

- Four-parameter fit (R2 = 0.60):
- Fold change = –93.2 dfull_sensor – 43.3 dactive_sensor – 22.1 dbinding_site – 9.4 dmin_target + 61.3

Definition of the parameters :
- dfull_sensor:  Ensamble defect for the full toehold switch sequence and structure 
- dactive_sensor: Ensemble defect was calculated directly from the sequence from the first base of the loop sequence. A completely single-stranded secondary structure was used for assessing design quality for dactive_sensor.
- dbinding_site: Ensemble defect was calculated in an analogous manner using the pairwise binding probabilities of the complete target RNA sequence and specifying a completely single-stranded ideal secondary structure in the binding site region.

#### Library functions
- Calculation of the minimum free energy (MFE) secondary structure of a singular RNA sequence
- NUPACK analysis

In [23]:
def DG(sequence,result_path,wait):
    file = open('{}pipo.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.close()
    final=[]
    semi_final=[]

    Popen(["mfe -T 29 {}pipo".format(result_path)],shell=True, stdout=PIPE)
    time.sleep(wait)
    lenght=len(sequence)
    with open("{}pipo.mfe".format(result_path)) as res:
        for r in res:
            r = r.strip('\n')
            if not r.startswith('%'):
                r = r.split('\t')
                semi_final.append(r)

    #final.append()                

    return (float(semi_final[2][0]))

    os.remove("{}pipo.mfe".format(result_path,))
    os.remove("{}pipo.in".format(result_path))
    
def complex_defect(sequence, secondary, result_path):
    file = open('{}toeh.in'.format(result_path), 'w')
    file.write("{}\n".format(sequence))
    file.write("{}".format(secondary))
    file.close()

    defect_toeh = 0
    count = 0
    with Popen(["complexdefect", "{}toeh".format(result_path)], stdout=PIPE) as proc:
        res = (proc.stdout.read()).decode("utf-8").split('\n')
        for l in res:
            count += 1
            if count == 16:
                defect_toeh = float(l)

    os.remove("{}toeh.in".format(result_path))
    return defect_toeh

def nupack_analysis(sequence, secondary_sensor,  window, sensor_type, result_path):
    list_for_table = []

    processed_sequence = sequence.upper().replace('T', 'U').replace(' ', '')
    reg_sequences = split_sequence(processed_sequence, window)
    rev_comp_sequences = [reversed_complement(s) for s in reg_sequences]

    if sensor_type == 'A':
        target_toehold_map = possible_toehold_A(reg_sequences, rev_comp_sequences)
    else:
        target_toehold_map = possible_toehold_B(reg_sequences, rev_comp_sequences)

    sequence = sequence.upper().replace('T', 'U')
    single_streadness_sequence = single_streadness(sequence, result_path, wait=6)
    for target, toehold in target_toehold_map.items():
        id = sequence.index(target)

        target_defect = sum(single_streadness_sequence[id:id+36])/36
        toehold_defect = sum(single_streadness(toehold, result_path)[0:30])/30
        sensor_defect = complex_defect(toehold, secondary_sensor, result_path)

        score = 5*(1-target_defect) + 4*(1-toehold_defect) + 3*sensor_defect

        list_for_table.append(tuple([target[0:36], toehold, 1-target_defect, 1-toehold_defect, sensor_defect, score]))

    return list_for_table

linear_Structure="............................................................"
linear_Structure_25="........................."
secondary_sensor_B = '.........................(((((((((((...(((((............)))))...))))))))))).....................'
secondary_2=".......................................(((((............)))))....................................+...................................."

#### Calculation of the thermodynamic parameters for each design

In [25]:


##fromNUPACK

dfull_sensor=[]

for i in range (len(df.Triggers)):
    dfull_sensor.append((complex_defect(df.iloc[i,2],secondary_sensor_B,"")))
dfull_sensor
P1=pd.Series(dfull_sensor)
df["dfull_sensor"]=P1.values

dactive_sensor=[]
for i in range (len(df.Triggers)):
   
    dactive_sensor.append((complex_defect(df.iloc[i,2][36::],linear_Structure,"")))
P2=pd.Series(dactive_sensor)
df["dactive_sensor"]=P2.values

dbinding_site=[]
for i in range (len(df.Triggers)):
   
    dbinding_site.append((complex_defect(df.iloc[i,2][0:25],linear_Structure_25,"")))
P3=pd.Series(dbinding_site)
df["dbinding_site"]=P3.values

Dg_RBS_linker=[]
for i in range(len(df.iloc[:,3])):
    RBS_linker=df.iloc[i,2][48:96]
    Dg_RBS_linker.append(DG(RBS_linker,"",2))
P5=pd.Series(Dg_RBS_linker)
df["Dg_RBS_linker"]=P5.values



/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [26]:
##fromNUPACK
score=[]
for i in range (len(df.Triggers)):
    score.append(54.3 -71.7*df.iloc[i,3] -49.1*df.iloc[i,4] -22.6**df.iloc[i,5])
P4=pd.Series(score)
df["score"]=P4.values

/home/anibal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
df

,Triggers,Sense,Toehold Switch,dfull_sensor,dactive_sensor,dbinding_site,Dg_RBS_linker,score
0,GGAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUC,Direct,GACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCCGGACUUUAGA...,0.1932,0.3884,0.1276,-10.477,19.888499
1,GAGUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCG,Direct,CGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACUCGGACUUUAGA...,0.2455,0.4085,0.1368,-10.816,15.108359
3,GUUUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAA,Direct,UUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAACGGACUUUAGA...,0.2200,0.3813,0.1308,-9.573,18.300622
5,UUGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUA,Direct,UAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAAGGACUUUAGA...,0.2264,0.4545,0.1300,-11.354,14.251367
6,UGGGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUAC,Direct,GUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCCAGGACUUUAGA...,0.2129,0.3511,0.1488,-7.812,20.205715
8,GGUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCC,Direct,GGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACCGGACUUUAGA...,0.2585,0.3807,0.4174,-10.302,13.398624
9,GUUAUGAUGGAUGGAGAUGAACAAGUCGAAUACCCA,Direct,UGGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUAACGGACUUUAGA...,0.2403,0.3774,0.4721,-9.571,14.182277
12,AUGAUGGAUGGAGAUGAACAAGUCGAAUACCCACUG,Direct,CAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUCAUGGACUUUAGA...,0.2388,0.3105,0.4128,-6.022,18.310260
14,GAUGGAUGGAGAUGAACAAGUCGAAUACCCACUGAA,Direct,UUCAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUCGGACUUUAGA...,0.2440,0.3581,0.4471,-5.853,15.191406
15,AUGGAUGGAGAUGAACAAGUCGAAUACCCACUGAAA,Direct,UUUCAGUGGGUAUUCGACUUGUUCAUCUCCAUCCAUGGACUUUAGA...,0.2288,0.3387,0.4287,-5.833,17.458541


## Save the output to a CSV

In [28]:
df.to_csv("Ranked_designs.csv")